In [7]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from denn import *
import pandas as pd
import seaborn as sns
import io #Used as buffer

In [8]:
nn_p=3
nn_w = 7
nn_tw = 1

# Functions

In [9]:
path= Path(f'../../data/cluster_results')
col_by_freq = True
col_palette = 'Set3'
pat = re.compile('.*/(exp\d)/(\w*)/nonn/freq([0-9\.]+)div(\w+)/(\w+)_\w+.csv')
decode_keys = ['experiment','function','freq','div','method']
# nn_pat = re.compile('.*/(exp\d)/(\w*)/nn/freq([0-9\.]+)\w+div([A-Za-z]+)/(\w+)_(\w+)_\w+.csv')
# nn_decode_keys = ['experiment','function','freq','div','method','replace_mech']

nn_pat = re.compile('.*/(exp\d)/(\w*)/nn/freq([0-9\.]+)nn_w(\d+)nn_p(\d+)\w+nn_tw(\d+)\w+div([A-Za-z]+)/(\w+)_(\w+)_\w+.csv')
nn_decode_keys = ['experiment','function','freq','nnw','nnp','nntw','div','method','replace_mech']#,


def get_nn_files(m): return list(path.glob(f'**/nn/**/*{m}.csv'))


def read_nn_csv(f,m):
    df = pd.read_csv(f)
    for k,v in zip(nn_decode_keys,nn_pat.search(str(f)).groups()): df[k] = v
    df['freq'] = df['freq'].astype(float)
    df['method'] = df['method'].str.replace('NNnorm_Worst', 'NN')
    df['method'] = df['method'] + '_' + df['div'] #'_' + df['replace_mech'] +
    
    df.drop(['replace_mech','div'], axis=1, inplace=True)
    df.rename({'0':m.upper(), m:m.upper()}, axis=1, inplace=True)
    return df

def get_data(m, normalize=False):
    nn_files = get_nn_files(m)
    data = pd.concat([read_nn_csv(f,m) for f in nn_files])
    data = data[data['nnw']==str(nn_w)]
    data = data[data['nntw']==str(nn_tw)]
    if normalize:
        data_norm = (data.groupby(['experiment','function','freq','method'])[m.upper()].mean().reset_index()
                         .groupby(['experiment','function'])[m.upper()].min().reset_index()
                         .rename({m.upper():m.upper()+'_norm'}, axis=1))
        data = data.merge(data_norm, 'left')
        data[m.upper()+'_norm'] = data[m.upper()] / data[m.upper()+'_norm']
    return data




# Table

In [10]:
df=get_data('nn_time', normalize=False)
df['function']=df['function'].str.title()
df.head()

,NN_TIME,experiment,function,freq,nnw,nnp,nntw,method
0,0.009165,exp1,Sphere,10.0,7,3,1,NN_Rst
1,0.009142,exp1,Sphere,10.0,7,3,1,NN_Rst
2,0.009135,exp1,Sphere,10.0,7,3,1,NN_Rst
3,0.009116,exp1,Sphere,10.0,7,3,1,NN_Rst
4,0.009136,exp1,Sphere,10.0,7,3,1,NN_Rst


In [11]:
table2=df.pivot_table(values='NN_TIME', index=['function','method'] ,columns=['experiment','freq'])
table2

experiment             exp1      exp2      exp3      exp4
freq                   10.0      10.0      10.0      10.0
function   method                                        
Rastrigin  NN_CwN  0.009578  0.009856  0.009684  0.009230
           NN_HMu  0.009306  0.009487  0.009543  0.009198
           NN_No   0.009141  0.009254  0.009189  0.009055
           NN_RI   0.009907  0.009766  0.009489  0.009234
Rosenbrock NN_CwN  0.009450  0.009515  0.009681  0.009178
           NN_HMu  0.013851  0.009377  0.009264  0.009376
           NN_No   0.009397  0.009167  0.009134  0.009179
           NN_RI   0.010431  0.010376  0.009831  0.009333
           NN_Rst  0.009208       NaN       NaN       NaN
Sphere     NN_CwN  0.009679  0.009479  0.009616  0.009239
           NN_HMu  0.009842  0.009313  0.012847  0.009324
           NN_No   0.009438  0.012559  0.009138  0.009018
           NN_RI   0.009533  0.009623  0.009504  0.009317
           NN_Rst  0.009106       NaN       NaN       NaN

In [12]:
print(table2.to_csv(float_format='%.3f'))

experiment,,exp1,exp2,exp3,exp4
freq,,10.0,10.0,10.0,10.0
function,method,,,,
Rastrigin,NN_CwN,0.010,0.010,0.010,0.009
Rastrigin,NN_HMu,0.009,0.009,0.010,0.009
Rastrigin,NN_No,0.009,0.009,0.009,0.009
Rastrigin,NN_RI,0.010,0.010,0.009,0.009
Rosenbrock,NN_CwN,0.009,0.010,0.010,0.009
Rosenbrock,NN_HMu,0.014,0.009,0.009,0.009
Rosenbrock,NN_No,0.009,0.009,0.009,0.009
Rosenbrock,NN_RI,0.010,0.010,0.010,0.009
Rosenbrock,NN_Rst,0.009,,,
Sphere,NN_CwN,0.010,0.009,0.010,0.009
Sphere,NN_HMu,0.010,0.009,0.013,0.009
Sphere,NN_No,0.009,0.013,0.009,0.009
Sphere,NN_RI,0.010,0.010,0.010,0.009
Sphere,NN_Rst,0.009,,,

